# 추천 시스템(Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링(content-based filtering)
  * 협업 필터링(collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

##Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [2]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 5.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633987 sha256=12a9858409dae6b6d3dc21a3ecdeca397fac26316321bae9c060cc5a7a84a1ad
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


###간단한 surprise 실습

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [4]:
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [5]:
model = SVD()

In [6]:
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9359  0.9415  0.9391  0.9385  0.9258  0.9362  0.0055  
MAE (testset)     0.7403  0.7410  0.7384  0.7412  0.7298  0.7381  0.0043  
Fit time          8.22    5.47    5.51    5.46    5.42    6.01    1.10    
Test time         0.36    0.16    0.21    0.22    0.15    0.22    0.07    


{'test_rmse': array([0.93586951, 0.9414989 , 0.93911366, 0.93853792, 0.92576351]),
 'test_mae': array([0.74026092, 0.74102105, 0.73842738, 0.74117568, 0.72977098]),
 'fit_time': (8.218984365463257,
  5.471323013305664,
  5.5051867961883545,
  5.457191228866577,
  5.42121434211731),
 'test_time': (0.35622549057006836,
  0.15606236457824707,
  0.20849943161010742,
  0.22466206550598145,
  0.15302395820617676)}

## 컨텐츠 기반 필터링(Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야하 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [7]:
import numpy as np
from surprise import Dataset

###나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기

In [10]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)


In [11]:
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [13]:
n_users = np.max(raw_data[: , 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [14]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = 1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [16]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    similarity = np.dot(my_vector, user_vector)
    if similarity > best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best match: 183, Best Match ID: 275


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1.and log2 > 0.:
    recommend_list.append(i)
  print(recommend_list)

###유클리드 거리를 사용해 추천
$$ euclidean = \sqrt{\sum^D_{d=1} (A_i - B_i)^2} $$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [18]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

print('Best match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best match: 14.832396974191326, Best Match ID: 737


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1.and log2 > 0.:
    recommend_list.append(i)
  print(recommend_list)

###코사인 유사도를 사용해 추천
$$ cos𝝷 = {\frac {A \cdot B}{||A|| \times ||B||}} $$
* 두 벡터가 이루고 있는 각을 계산

In [20]:
def compute_cos_similarity(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)
  return dot / (norm1 * norm2)

In [26]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos__similarity = compute_cos_similarity(my_vector,user_vector)
    if cos__similarity > best_match:
      best_match = cos__similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best match: 0.569065731527988, Best Match ID: 915


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1.and log2 > 0.:
    recommend_list.append(i)
  print(recommend_list)

###기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [28]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [29]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

print('Best match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best match: 55.06359959174482, Best Match ID: 737


In [30]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos__similarity = compute_cos_similarity(my_vector,user_vector)
    if cos__similarity > best_match:
      best_match = cos__similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best match: 0.569065731527988, Best Match ID: 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요없다
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에서 나오지 않은 항목은 임베딩을 만들수 없음
    * 추가 특성을 사용하기 어려움

In [31]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [32]:
data = Dataset.load_builtin('ml-100k', prompt=False)

###KNN 을 사용한 협업 필터링

In [34]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9769  0.9829  0.9775  0.9740  0.9837  0.9790  0.0037  
MAE (testset)     0.7726  0.7740  0.7734  0.7689  0.7754  0.7729  0.0022  
Fit time          0.53    0.97    0.95    0.73    0.59    0.76    0.18    
Test time         6.37    7.03    7.56    6.05    4.09    6.22    1.19    


{'test_rmse': array([0.97690596, 0.98286012, 0.97750081, 0.9740374 , 0.98369257]),
 'test_mae': array([0.77262371, 0.77400371, 0.77343892, 0.7689057 , 0.775414  ]),
 'fit_time': (0.5306000709533691,
  0.9713590145111084,
  0.9538333415985107,
  0.7334034442901611,
  0.5888690948486328),
 'test_time': (6.37154483795166,
  7.032374620437622,
  7.562734127044678,
  6.049937725067139,
  4.0882697105407715)}

###SVD를 사용한 협업 

In [36]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9376  0.9369  0.9291  0.9392  0.9385  0.9363  0.0037  
MAE (testset)     0.7396  0.7402  0.7323  0.7396  0.7388  0.7381  0.0029  
Fit time          14.20   17.25   18.00   15.63   10.80   15.18   2.55    
Test time         0.67    0.62    0.39    0.26    0.17    0.42    0.20    


{'test_rmse': array([0.93758389, 0.93689831, 0.92910521, 0.93921621, 0.93848718]),
 'test_mae': array([0.7396073 , 0.74017416, 0.73233948, 0.7395587 , 0.73882114]),
 'fit_time': (14.203165054321289,
  17.24929451942444,
  17.99697971343994,
  15.629151821136475,
  10.796850204467773),
 'test_time': (0.670914888381958,
  0.6200985908508301,
  0.3907458782196045,
  0.26491546630859375,
  0.16986441612243652)}

###NMF 를 사용한 협업 필터링

In [35]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9556  0.9553  0.9635  0.9707  0.9633  0.9617  0.0057  
MAE (testset)     0.7534  0.7499  0.7569  0.7654  0.7537  0.7559  0.0052  
Fit time          14.37   17.25   16.35   13.31   9.29    14.11   2.79    
Test time         0.63    0.42    0.34    0.23    0.16    0.36    0.16    


{'test_rmse': array([0.95564022, 0.95529682, 0.96348254, 0.97067741, 0.96327107]),
 'test_mae': array([0.75335545, 0.74994698, 0.75690057, 0.76538738, 0.7537344 ]),
 'fit_time': (14.365390539169312,
  17.254948616027832,
  16.349149227142334,
  13.306716680526733,
  9.293825626373291),
 'test_time': (0.6317675113677979,
  0.41789770126342773,
  0.3367455005645752,
  0.23322153091430664,
  0.15727639198303223)}

###SVD++ 를 사용한 협업 필터링

In [48]:
#model = SVDpp()
#cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

## 하이브리드 (Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

### 기본

In [72]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import  Dataset

In [105]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [106]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [107]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

In [108]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [109]:
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


In [110]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [111]:
np.matmul(np.matmul(U, S), V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

####사용자 기반 추천

* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [91]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos__similarity = compute_cos_similarity(my_vector,user_vector)
    if cos__similarity > best_match:
      best_match = cos__similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best match: 0.9999942295956324, Best Match ID: 235


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1.and log2 > 0.:
    recommend_list.append(i)
  print(recommend_list)

####항목 기반 추천

* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [112]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
  if my_id != user_id:
    cos__similarity = compute_cos_similarity(my_vector,user_vector)
    if cos__similarity > best_match:
      best_match = cos__similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best match: 0.9999999951364144, Best Match ID: 1287


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
  print(recommend_list)

###비음수 행렬 분해를 사용한 하이브리드 추천


In [115]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [116]:
A, B, iter = non_negative_factorization(adj_matrix, n_components=2)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [117]:
np.matmul(A, B)

array([[3.71108858e+00, 1.48454102e+00, 7.39535363e-01, ...,
        3.64490531e-03, 1.45506510e-02, 1.44110916e-02],
       [2.11724416e+00, 2.37338725e-01, 5.51650703e-01, ...,
        4.76092332e-03, 3.03585166e-05, 0.00000000e+00],
       [9.85272360e-01, 1.10447010e-01, 2.56713988e-01, ...,
        2.21552443e-03, 1.41275191e-05, 0.00000000e+00],
       ...,
       [1.04485667e+00, 1.17126289e-01, 2.72238757e-01, ...,
        2.34950819e-03, 1.49818804e-05, 0.00000000e+00],
       [1.45758857e+00, 5.42112787e-01, 2.99181695e-01, ...,
        1.61177659e-03, 5.15921394e-03, 5.10771005e-03],
       [2.44733103e+00, 9.41208708e-01, 4.95740038e-01, ...,
        2.56992125e-03, 9.08287324e-03, 8.99389282e-03]])

####사용자 기반 추천

In [118]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos__similarity = compute_cos_similarity(my_vector,user_vector)
    if cos__similarity > best_match:
      best_match = cos__similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best match: 0.9999942295956324, Best Match ID: 235


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1.and log2 > 0.:
    recommend_list.append(i)
  print(recommend_list)

####항목 기반 추천

In [120]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
  if my_id != user_id:
    cos__similarity = compute_cos_similarity(my_vector,user_vector)
    if cos__similarity > best_match:
      best_match = cos__similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best match: 0.9999999951364144, Best Match ID: 1287


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
  print(recommend_list)